# US county-level data
* Get everything in shape from NYT
* Then add in JHU for recent observations

TO DO
make notebook into script
use general function that calls sub-functions
upload csv to ESRI, get item ID
create state aggregate columns

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
df = pd.read_csv('s3://public-health-dashboard/jhu_covid19/county_time_series_330.csv')

In [3]:
df['date'] = pd.to_datetime(df.date)

In [4]:
df[(df.state=="California") & (df.date >= '3/30/20')].sort_values(['date'])

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths
1438,Alameda,California,6001.0,2020-03-30,NaN,NaN,304,7,NaN,NaN,13811,279
2303,Santa Clara,California,6085.0,2020-03-30,NaN,NaN,848,29,NaN,NaN,13811,279
2243,Santa Barbara,California,6083.0,2020-03-30,NaN,NaN,68,0,NaN,NaN,13811,279
2242,Santa Barbara,California,6083.0,2020-03-30,NaN,NaN,88,0,NaN,NaN,13811,279
2226,San Mateo,California,6081.0,2020-03-30,NaN,NaN,309,6,NaN,NaN,13811,279
...,...,...,...,...,...,...,...,...,...,...,...,...
1625,Kings,California,6031.0,2020-03-30,NaN,NaN,3,0,NaN,NaN,13811,279
1621,Kern,California,6029.0,2020-03-30,NaN,NaN,51,1,NaN,NaN,13811,279
1620,Kern,California,6029.0,2020-03-30,NaN,NaN,76,1,NaN,NaN,13811,279
1765,Mendocino,California,6045.0,2020-03-30,NaN,NaN,4,0,NaN,NaN,13811,279


In [5]:
df[(df.state=="California") & (df.date == '3/30/20') & (df.county=="Los Angeles")]

,county,state,fips,date,Lat,Lon,cases,deaths,incident_rate,people_tested,state_cases,state_deaths
1691,Los Angeles,California,6037.0,2020-03-30,NaN,NaN,2474,44,NaN,NaN,13811,279
1692,Los Angeles,California,6037.0,2020-03-30,NaN,NaN,2147,37,NaN,NaN,13811,279


### Functions to be used

In [ ]:
def coerce_fips_integer(df):
    def integrify(x):
        return int(float(x)) if not pd.isna(x) else None

    cols = [
        "fips",
    ]
    
    new_cols = {c: df[c].apply(integrify, convert_dtype=False) for c in cols}
    
    return df.assign(**new_cols)

In [ ]:
def correct_county_fips(row):
    if len(str(row.fips)) == 5:
        return str(row.fips)
    elif row.fips is not None:
        return "0" + str(row.fips)
    elif row.fips is None:
        return ""

## Use NYT for county-level time-series data

In [ ]:
bucket_name = "public-health-dashboard"
county = pd.read_csv(f"s3://{bucket_name}/jhu_covid19/county_time_series_330.csv")

## JHU data that needs to be a DAG

* Read in feature layer
* Add date column
* Apply clean_jhu_county function
* Do upsert

In [ ]:
# First, we need to make sure our nyt_geog crosswalk is open
NYT_330_COMMIT = "99b30cbf4181e35bdcc814e2b29671f38d7860a7"
NYT_COUNTY_URL = (
    f"https://raw.githubusercontent.com/nytimes/covid-19-data/{NYT_330_COMMIT}/"
    "us-counties.csv"
)
county = pd.read_csv(NYT_COUNTY_URL)

nyt_geog = county[county.fips.notna()][['fips', 'county', 'state']].drop_duplicates()

nyt_geog = coerce_fips_integer(nyt_geog)
nyt_geog["fips"] = nyt_geog.apply(correct_county_fips, axis=1)

In [ ]:
# Read in JHU feature layer in
jhu = 

In [ ]:
def clean_jhu_county(df):
    # Only keep certain columns and rename them to match NYT schema
    keep_cols = [
        "Province_State",
        "Country_Region",
        "Lat",
        "Long_",
        "Confirmed",
        "Deaths",
        "FIPS",
        "Incident_Rate",
        "People_Tested",
        "date",
    ]

    df = df[keep_cols]

    df.rename(
        columns={
            "Confirmed": "cases",
            "Deaths": "deaths",
            "FIPS": "fips",
            "Long_": "Lon",
            "People_Tested": "people_tested",
            "Incident_Rate": "incident_rate",
        },
        inplace=True,
    )

    # Use FIPS to merge in NYT columns for county and state names
    # There are some values with no FIPS, NYT calls these county = "Unknown"
    df = pd.merge(df, nyt_geog, on="fips", how="left", validate="m:1")

    # Fix when FIPS is unknown, which wouldn't have merged in anything from nyt_geog
    df["county"] = df.apply(
        lambda row: "Unknown" if row.fips is None else row.county, axis=1
    )
    df["state"] = df.apply(
        lambda row: row.Province_State if row.fips is None else row.state, axis=1
    )
    df["fips"] = df.fips.fillna("")

    # Only keep certain columns and rename them to match NYT schema
    drop_cols = ["Province_State", "Country_Region"]

    df = df.drop(columns=drop_cols)

    return df

In [ ]:
# Pretend 3/27 is the current date showing for JHU
jhu_today = clean_jhu_county(jhu)

In [ ]:
def sort_drop_duplicates(df):
    for col in ["Lat", "Lon"]:
        df[col] = df.groupby(["fips", "county", "state"])[col].transform("max")

    # Sort columns
    col_order = [
        "county",
        "state",
        "fips",
        "date",
        "Lat",
        "Lon",
        "cases",
        "deaths",
        "incident_rate",
        "people_tested",
    ]

    df = df.reindex(columns=col_order).sort_values(
        ["state", "county", "fips", "date", "cases"]
    )

    # Set data types for cases and deaths? Seems ok for now....
    for col in ["incident_rate", "people_tested"]:
        df[col] = df[col].astype(float)

    # Drop duplicates
    # Either: (1) values are updated throughout the day, or
    # (2) slight discrepancies between NYT and JHU.
    # Regardless, take the max value for cases and deaths for each date.
    group_cols = ["state", "county", "fips", "date"]
    for col in ["cases", "deaths"]:
        df[col] = df.groupby(group_cols).transform("max")

    df = df.drop_duplicates(subset=group_cols)

    return df

In [ ]:
jhu_today = sort_drop_duplicates(jhu_today)

In [ ]:
# Append everything just once -- should this step happen?
us_county = county.append(jhu_today, sort=False)

# Save as temporary file?

In [ ]:
# Now it's ready to be upserted
# Also, keep Ian's localize then UTC timezone stuff